In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('../raw_data/ml-25m/movies.csv')
ratings = pd.read_csv('../raw_data/ml-25m/ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
dt = pd.merge(movies, ratings, how='inner', on='movieId')
dt.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5,1141415820
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0,1573944252
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,858625949
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,890492517


In [6]:
ratings = dt[['userId', 'movieId', 'rating']].copy()
ratings.head()

,userId,movieId,rating
0,2,1,3.5
1,3,1,4.0
2,4,1,3.0
3,5,1,4.0
4,8,1,4.0


In [7]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
movie_c = CategoricalDtype(sorted(ratings.movieId.unique()), ordered=True)
user_c = CategoricalDtype(sorted(ratings.userId.unique()), ordered=True)
row = ratings.userId.astype(user_c).cat.codes
col = ratings.movieId.astype(movie_c).cat.codes
sparse_matrix = csr_matrix((ratings["rating"], (row, col)),shape=(user_c.categories.size, movie_c.categories.size))
df = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, index=user_c.categories, columns=movie_c.categories)

In [8]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,209145,209147,209151,209153,209155,209157,209159,209163,209169,209171
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.decomposition import PCA
pca = PCA()

# find all Principal Components PC (no y!)
pca.fit(sparse_matrix)

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=100)
km.fit(sparse_matrix)

In [ ]:
km.labels_

In [ ]:
df_head = df.head(10).copy()

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df_head[:,0], df_head[:,1], c=km.labels_)
plt.title('KMeans clustering'); plt.xlabel('PC 1'); plt.ylabel('PC 2')